In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
# df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
df = df.drop(columns = 'id')
#df_test = df_test.drop(columns = 'id')

In [ ]:
x = df.drop(columns = 'target')
y = df.target

In [ ]:
##create folds
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df,y=df.target.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold

## Ensemble

### Train.py

In [ ]:
# params ={'device':['gpu'],'learning_rate': 0.1508067112491576, 'lambda_l1': 2.4697757781805905, 'lambda_l2': 9.083741442132819e-08, 'num_leaves': 23, 'feature_fraction': 0.6677259987024685, 'bagging_fraction': 0.6000000000000001, 'subsample': 0.8, 'min_child_samples': 35}

In [ ]:
import joblib
import lightgbm as lgb
import xgboost as xg
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    df_train = df_train.drop(columns = 'kfold')
    df_valid = df_valid.drop(columns = 'kfold')
    x_train = df_train.drop('target', axis=1).values
    y_train = df_train.target.values
    x_valid = df_valid.drop('target', axis=1).values
    y_valid = df_valid.target.values
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict_proba(x_valid)[:,1]
    roc_auc_score = metrics.roc_auc_score(y_valid,y_pred)
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    File_name = 'model_lgb' + str(fold)
    joblib.dump(
    clf,File_name)
for i in range(5):
    run(fold = i)

In [ ]:
model_0_xgb= joblib.load('./model_lgb0')
model_1_xgb =joblib.load('./model_lgb1')
model_2_xgb= joblib.load('./model_lgb2')
model_3_xgb= joblib.load('./model_lgb3')
model_4_xgb= joblib.load('./model_lgb4')

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_test = df_test.drop(columns = 'id')

In [ ]:
y_final_3 = model_3_xgb.predict_proba(df_test)[:,1]
y_final_0 = model_0_xgb.predict_proba(df_test)[:,1]
y_final_1 = model_1_xgb.predict_proba(df_test)[:,1]
y_final_2 = model_2_xgb.predict_proba(df_test)[:,1]
y_final_4 = model_4_xgb.predict_proba(df_test)[:,1]

In [ ]:
y_final_avg = (y_final_0 + y_final_1 +y_final_2 + y_final_3 + y_final_4)/5

In [ ]:
submission['target'] = y_final_avg 

In [ ]:
submission.to_csv('pred_csv_avg.csv',index = False)